In [1]:
!pip install -q -U transformers accelerate datasets trl evaluate rouge-score sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 92.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.1 MB/s eta 0:00:00

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [12]:
from dataclasses import dataclass, asdict
from typing import Optional, List, Dict, Any
import os, json, random, math, re
from pathlib import Path

import numpy as np
import torch
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling

from trl import SFTTrainer, SFTConfig

import evaluate

In [4]:
# Cell 8 cũ của bạn đang thiếu đoạn login
from kaggle_secrets import UserSecretsClient
import wandb
import os

user_secrets = UserSecretsClient()

# 1. Lấy key từ Secrets (đảm bảo bạn đã Add Secret tên WANDB_API_KEY bên thanh phải)
my_key = user_secrets.get_secret("WANDB_API_KEY")

# 2. ĐĂNG NHẬP NGAY TẠI ĐÂY (Quan trọng)
wandb.login(key=my_key)
os.environ["WANDB_API_KEY"] = my_key
os.environ["WANDB_PROJECT"] = "qwen3-qa-vietnamese" # Đặt tên project cho dễ quản lý

print("✅ Đã đăng nhập WandB thành công! Giờ bạn có thể train.")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 52200189 (52200189-ton-duc-thang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Đã đăng nhập WandB thành công! Giờ bạn có thể train.


# Configuration

In [5]:
@dataclass
class QAConfig:
    # Đường dẫn dữ liệu & thư mục output
    split_dir: str = "/kaggle/input/qa-splits-fixed"
    output_dir: str = "/kaggle/working/qwen3_qa_model"

    # Model name
    base_model: str = "Qwen/Qwen3-0.6B"


    seed: int = 42

    # Cấu hình token / model
    model_max_len: int = 512

    # Hyperparams train
    num_train_epochs: int = 2
    per_device_train_batch_size: int = 8
    gradient_accumulation_steps: int = 2
    learning_rate: float = 5e-5
    weight_decay: float = 0.0
    warmup_ratio: float = 0.03

    logging_steps: int = 100
    save_strategy: str = "epoch"
    eval_strategy: str = "epoch"
    save_total_limit = 1

    load_best_model_at_end: bool = True
    metric_for_best_model: str = "eval_loss"
    greater_is_better: bool = False
    gradient_checkpointing: bool = True
    fp16: bool = True
    save_safetensors: bool = True  # lưu model.safetensors


cfg = QAConfig()
os.makedirs(cfg.output_dir, exist_ok=True)

print("=== QAConfig ===")
print(json.dumps(asdict(cfg), indent=2, ensure_ascii=False))

=== QAConfig ===
{
  "split_dir": "/kaggle/input/qa-splits-fixed",
  "output_dir": "/kaggle/working/qwen3_qa_model",
  "base_model": "Qwen/Qwen3-0.6B",
  "seed": 42,
  "model_max_len": 512,
  "num_train_epochs": 2,
  "per_device_train_batch_size": 8,
  "gradient_accumulation_steps": 2,
  "learning_rate": 5e-05,
  "weight_decay": 0.0,
  "warmup_ratio": 0.03,
  "logging_steps": 100,
  "save_strategy": "epoch",
  "eval_strategy": "epoch",
  "load_best_model_at_end": true,
  "metric_for_best_model": "eval_loss",
  "greater_is_better": false,
  "gradient_checkpointing": true,
  "fp16": true,
  "save_safetensors": true
}


# Pre-Processing

In [6]:
PROMPT_TEMPLATE_VI = (
    "Bạn là một trợ lý AI hữu ích, trả lời câu hỏi dựa trên ngữ cảnh cho sẵn.\n\n"
    "### Ngữ cảnh:\n{context}\n\n"
    "### Câu hỏi:\n{question}\n\n"
    "### Trả lời:\n"
)


class VietnamesePreprocessor:
    """
    Xử lý tiền xử lý đơn giản cho tiếng Việt:
    - Chuẩn hóa khoảng trắng
    - Giữ nguyên dấu tiếng Việt
    - Có thể mở rộng thêm nếu cần
    """
    @staticmethod
    def normalize(text: str) -> str:
        if text is None:
            return ""
        # bỏ khoảng trắng thừa, giữ nguyên dấu
        text = str(text).replace("\u00a0", " ")  # non‑breaking space
        text = re.sub(r"\s+", " ", text).strip()
        return text

    def build_prompt(self, context: str, question: str) -> str:
        context = self.normalize(context)
        question = self.normalize(question)
        return PROMPT_TEMPLATE_VI.format(context=context, question=question)



class QADatasetBuilder:
    """
    - Đọc file JSON format: [{context, question, answer}, ...]
    - Tiền xử lý tiếng Việt
    - Build prompt & target
    - Chia train/val/test theo tỉ lệ 8:1:1
    """
    def __init__(self, cfg: QAConfig, preprocessor: Optional[VietnamesePreprocessor] = None):
        self.cfg = cfg
        self.preproc = preprocessor or VietnamesePreprocessor()
        self.tokenizer = AutoTokenizer.from_pretrained(cfg.base_model)

    def load_raw(self) -> List[Dict[str, Any]]:
        path = Path(self.cfg.dataset_path)
        if not path.exists():
            raise FileNotFoundError(f"Không tìm thấy file dataset tại: {path}")
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if not isinstance(data, list):
            raise ValueError("qa_dataset.json cần có format: list các object {context, question, answer}.")
        return data
    
        return errors
    def validate_item(self, prompt, answer, tokenizer, max_len):
        errors = []
    
        # 1. answer rỗng
        if not answer.strip():
            errors.append("empty_answer")
    
        # 2. prompt phải kết thúc đúng format
        if not prompt.endswith("### Trả lời:\n"):
            errors.append("bad_prompt_suffix")
    
        # 3. tokenize full
        full_ids = tokenizer(prompt + answer, add_special_tokens=True, truncation=False)["input_ids"]
        if len(full_ids) == 0:
            errors.append("empty_tokenized_text")
    
        # 4. quá dài
        if len(full_ids) > max_len:
            errors.append("sequence_too_long")
    
        # 5. ít nhất phải có 1 token answer
        prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
        if len(full_ids) <= len(prompt_ids):
            errors.append("no_answer_tokens")
    
        return errors

    def preprocess(self, raw_data: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        items = []
        for ex in raw_data:
            context = ex.get("context", "")
            question = ex.get("question", "")
            answer = ex.get("answer", "")

            question = self.preproc.normalize(question)
            answer = self.preproc.normalize(answer)
            context = self.preproc.normalize(context)
            
            if not question or not answer:
                continue
            
            prompt = self.preproc.build_prompt(context=context, question=question)
            errors = self.validate_item(prompt, answer, self.tokenizer, self.cfg.model_max_len)
            if errors:
                print(f"⚠️ BỎ SAMPLE (lỗi={errors}): Q={question} | A={answer[:50]}")
                continue
            items.append({
                "text": prompt + answer,
                "target": answer
            })



        if len(items) < 2:
            print("Cảnh báo: Dataset sau tiền xử lý rất nhỏ, hãy bổ sung thêm dữ liệu.")
        return items


    def load_fixed_splits(self) -> DatasetDict:
        """
        Load bộ split cố định train/val/test từ JSONL
        (do bạn tạo bằng baseline splitting trước đó)
        """
        base = "/kaggle/input/qa-splits-fixed"
        train_path = f"{base}/train.jsonl"
        val_path   = f"{base}/val.jsonl"
        test_path  = f"{base}/test.jsonl"

        if not all(os.path.exists(p) for p in [train_path, val_path, test_path]):
            raise FileNotFoundError("❌ Không tìm thấy bộ split cố định train/val/test!")

        def load_jsonl(path):
            rows = []
            with open(path, "r", encoding="utf-8") as f:
                for line in f:
                    rows.append(json.loads(line))
            return rows

        print("📥 Đang load bộ split cố định train/val/test...")

        train_raw = load_jsonl(train_path)
        val_raw   = load_jsonl(val_path)
        test_raw  = load_jsonl(test_path)

        print(f"Train: {len(train_raw)}, Val: {len(val_raw)}, Test: {len(test_raw)}")

        # PREPROCESS từng mẫu → build prompt
        train_proc = self.preprocess(train_raw)
        val_proc   = self.preprocess(val_raw)
        test_proc  = self.preprocess(test_raw)

        return DatasetDict({
            "train": Dataset.from_list(train_proc),
            "validation": Dataset.from_list(val_proc),
            "test": Dataset.from_list(test_proc)
        })
    

    def build(self) -> DatasetDict:
        return self.load_fixed_splits()

# Build Decoder Only Transformer

In [7]:
class QwenDecoderOnlyQA:
    """
    Wrapper cho model decoder‑only Qwen3‑0.6B dùng cho Q&A tiếng Việt.
    Đơn giản hóa: không dùng QLoRA, không 4‑bit để train nhanh hơn.
    """
    def __init__(self, cfg: QAConfig):
        self.cfg = cfg
        self.tokenizer = None
        self.model = None

    def load_tokenizer_and_model(self):
        print("Load tokenizer & model:", self.cfg.base_model)
        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg.base_model)
        self.tokenizer.padding_side = "left"
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(
            self.cfg.base_model,
            device_map="auto",
            torch_dtype=torch.float32,
        )
        

        self.tokenizer.model_max_length = self.cfg.model_max_len
        self.model.config.pad_token_id = self.tokenizer.pad_token_id
        self.model.config.use_cache = False  # để hỗ trợ gradient checkpoint

        return self.tokenizer, self.model

    @staticmethod
    def format_sample(example: Dict[str, str]) -> str:
        # SFTTrainer chỉ cần text hoàn chỉnh
        return example["text"]

    def build_trainer(self, ds: DatasetDict) -> SFTTrainer:
        if self.tokenizer is None or self.model is None:
            self.load_tokenizer_and_model()

        # --- CẤU HÌNH SFTConfig (Tất cả tham số dồn vào đây) ---
        sft_args = SFTConfig(
            output_dir=self.cfg.output_dir,
            num_train_epochs=self.cfg.num_train_epochs,
            per_device_train_batch_size=self.cfg.per_device_train_batch_size,
            gradient_accumulation_steps=self.cfg.gradient_accumulation_steps,
            learning_rate=self.cfg.learning_rate,
            weight_decay=self.cfg.weight_decay,
            warmup_ratio=self.cfg.warmup_ratio,
            logging_steps=self.cfg.logging_steps,
            save_strategy=self.cfg.save_strategy,
            eval_strategy=self.cfg.eval_strategy,
            load_best_model_at_end=self.cfg.load_best_model_at_end,
            metric_for_best_model=self.cfg.metric_for_best_model,
            greater_is_better=self.cfg.greater_is_better,
            save_safetensors=self.cfg.save_safetensors,
            dataset_text_field="text",
            # --- CÁC THAM SỐ QUAN TRỌNG CHO GPU & TRL 0.25.1 ---
            gradient_checkpointing=False, # Tiết kiệm VRAM
            dataset_kwargs={
                "max_seq_length": self.cfg.model_max_len,
                "packing": False,
            },                  # Độ dài chuỗi (256)
            packing=False,                                          # Tắt packing để tránh lỗi
            disable_tqdm=False,                                     # Hiện thanh tiến trình
            fp16=True,                                              # Ép dùng FP16 cho T4/P100
            bf16=False,                                             # Tắt BF16
            dataloader_num_workers=2,
        )

        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,
        )

        # --- KHỞI TẠO TRAINER (Gọn nhẹ, không truyền tham số thừa) ---
        trainer = SFTTrainer(
            model=self.model,
            args=sft_args,
            train_dataset=ds["train"],
            eval_dataset=ds["validation"],
            formatting_func=self.format_sample,
            data_collator=data_collator,
            processing_class=self.tokenizer,
        )

        print("SFT hyperparameters:")
        print(sft_args.to_dict())
        return trainer

        print("SFT hyperparameters:")
        print(sft_args.to_dict())
        return trainer

# Training Pipeline

In [8]:
class QwenQAPipeline:
    """
    Pipeline OOP gọn cho toàn bộ bài toán:
    - Tiền xử lý & chia dữ liệu 6:2:2
    - Build trainer SFT
    - Train + Evaluate + Lưu best model (safetensors hoặc pytorch_model.bin)
    """
    def __init__(self, cfg: QAConfig):
        self.cfg = cfg
        self.dataset_builder = QADatasetBuilder(cfg)
        self.model_wrapper = QwenDecoderOnlyQA(cfg)

        self.ds: Optional[DatasetDict] = None
        self.trainer: Optional[SFTTrainer] = None

    def prepare(self):
        print("=== Bước 1: Chuẩn bị dữ liệu (preprocess) ===")
        self.ds = self.dataset_builder.build()

        print("\n=== Bước 2: Load model & build SFTTrainer ===")
        self.trainer = self.model_wrapper.build_trainer(self.ds)

    def train(self):
        if self.trainer is None or self.ds is None:
            self.prepare()

        print("\n=== Bước 3: Train SFT ===")
        train_result = self.trainer.train()
        metrics = getattr(train_result, "metrics", {})
        metrics["train_samples"] = len(self.ds["train"])

        # Sau khi train xong, Trainer sẽ giữ best model (nhờ load_best_model_at_end=True)
        # => Lưu model tốt nhất ra output_dir
        self.save_best_model()

        # Lưu metrics train
        train_metrics_path = os.path.join(self.cfg.output_dir, "train_metrics.json")
        with open(train_metrics_path, "w", encoding="utf-8") as f:
            json.dump({k: float(v) if isinstance(v, (int, float)) else v for k, v in metrics.items()},
                      f, indent=2, ensure_ascii=False)
        print(f"Đã lưu train_metrics vào: {train_metrics_path}")
        return metrics

    def save_best_model(self):
        # trainer.save_model sẽ lưu model hiện tại (đã là best nếu load_best_model_at_end=True)
        print("\n=== Bước 4: Lưu best model ===")
        self.trainer.save_model(self.cfg.output_dir)
        self.model_wrapper.tokenizer.save_pretrained(self.cfg.output_dir)

        # Với save_safetensors=True, thư mục output sẽ có file:
        #   - model.safetensors (hoặc adapter_model.safetensors nếu dùng PEFT)
        # Nếu không, sẽ có pytorch_model.bin
        print("Đã lưu model & tokenizer vào:", self.cfg.output_dir)

    def _prepare_split_for_sft(self, split: str):
        """
        Dùng lại logic của SFTTrainer để biến self.ds[split]
        từ dạng raw (text/target) -> dạng tokenized (input_ids, labels, ...).
        """
        if self.trainer is None or self.ds is None:
            raise RuntimeError("Cần gọi prepare()/train() trước khi _prepare_split_for_sft.")
        if split not in self.ds:
            raise ValueError(f"Tập '{split}' không tồn tại trong DatasetDict.")

        raw_ds = self.ds[split]

        # Lấy cấu hình SFT/TrainingArguments từ trainer
        sft_args = self.trainer.args

        # Tokenizer và formatting_func dùng khi build SFTTrainer
        tokenizer = self.model_wrapper.tokenizer
        formatting_func = getattr(self.model_wrapper, "format_sample", None)

        # Giá trị packing giống train/eval
        packing = getattr(sft_args, "packing", False)

        processed_ds = self.trainer._prepare_dataset(
            dataset=raw_ds,
            processing_class=tokenizer,
            args=sft_args,
            packing=packing,
            formatting_func=formatting_func,
            dataset_name=split,
        )
        return processed_ds

    def evaluate_split(self, split: str = "test") -> Dict[str, float]:
        """
        Evaluate trên một split bất kỳ của DatasetDict.
        - Dùng SFTTrainer.evaluate trên dataset đã được _prepare_split_for_sft
          để tính loss / perplexity.
        - Đồng thời dùng raw dataset (text/target) để generate vài mẫu và tính BLEU.
        """
        if self.trainer is None or self.ds is None:
            raise RuntimeError("Cần gọi prepare()/train() trước khi evaluate.")

        if split not in self.ds:
            raise ValueError(f"Tập '{split}' không tồn tại trong DatasetDict.")

        print(f"\n=== Bước 5: Evaluate trên tập {split} ===")

        # Dataset raw với các field 'text', 'target'
        raw_eval_ds = self.ds[split]

        # Dataset đã được token hóa cho SFTTrainer
        tokenized_eval_ds = self._prepare_split_for_sft(split)

        # Gọi evaluate của trainer trên dataset đã token hóa
        eval_metrics = self.trainer.evaluate(eval_dataset=tokenized_eval_ds)

        # Tính thêm perplexity nếu có eval_loss
        if "eval_loss" in eval_metrics:
            try:
                eval_metrics["perplexity"] = math.exp(eval_metrics["eval_loss"])
            except OverflowError:
                eval_metrics["perplexity"] = float("inf")

        # Thêm BLEU đơn giản để tham khảo
        bleu = evaluate.load("sacrebleu")

        preds = []
        refs = []

        # Chỉ lấy tối đa 50 mẫu để generate cho nhanh
        for ex in raw_eval_ds.select(range(min(50, len(raw_eval_ds)))):
            text = ex["text"]
            refs.append([ex["target"]])

            # Tạo prompt đầu vào đến phần '### Trả lời:'
            m = re.search(r"### Trả lời:\n(.*)$", text, flags=re.DOTALL)
            prompt = text.split("### Trả lời:")[0] + "### Trả lời:\n"

            inputs = self.model_wrapper.tokenizer(prompt, return_tensors="pt").to(self.model_wrapper.model.device)
            with torch.no_grad():
                out = self.model_wrapper.model.generate(
                    **inputs,
                    max_new_tokens=128,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    pad_token_id=self.model_wrapper.tokenizer.pad_token_id,
                    eos_token_id=self.model_wrapper.tokenizer.eos_token_id,
                )
            # decoded = self.model_wrapper.tokenizer.decode(
            #     out[0][inputs["input_ids"].shape[1]:],
            #     skip_special_tokens=True
            # )
            # preds.append(decoded.strip())
            full = self.model_wrapper.tokenizer.decode(out[0], skip_special_tokens=True)
            pred_only = full.split("### Trả lời:")[-1].strip()
            preds.append(pred_only)


        bleu_res = bleu.compute(predictions=preds, references=refs)
        eval_metrics["bleu"] = float(bleu_res.get("score", 0.0))

        eval_path = os.path.join(self.cfg.output_dir, f"eval_{split}.json")
        with open(eval_path, "w", encoding="utf-8") as f:
            json.dump(
                {k: float(v) if isinstance(v, (int, float)) else v for k, v in eval_metrics.items()},
                f,
                indent=2,
                ensure_ascii=False,
            )
        print(f"Đã lưu eval_metrics vào: {eval_path}")
        return eval_metrics

In [9]:
import trl
import transformers
import torch

print(f"TRL version: {trl.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Torch version: {torch.__version__}")

TRL version: 0.25.1
Transformers version: 4.57.1
Torch version: 2.6.0+cu124


# Main chạy thực nghiệm

In [10]:
# ===== MAIN PIPELINE =====

# Truyền tham số epoch
EPOCHS = 2
cfg.num_train_epochs = EPOCHS

pipeline = QwenQAPipeline(cfg)

# Chuẩn bị dữ liệu + trainer
pipeline.prepare()

# Train
train_metrics = pipeline.train()
print("\n=== Train metrics ===")
print(train_metrics)

# Evaluate nhanh trên tập test (đã chia 20%)
test_metrics = pipeline.evaluate_split(split="test")
print("\n=== Test metrics ===")
print(test_metrics)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

=== Bước 1: Chuẩn bị dữ liệu (preprocess) ===
📥 Đang load bộ split cố định train/val/test...
Train: 20298, Val: 2256, Test: 2507
⚠️ BỎ SAMPLE (lỗi=['sequence_too_long']): Q=Các xóm nào được liệt kê trong xã Khánh Thiện? | A=Các xóm được liệt kê bao gồm: 1, 1 Đông Cường, 1 N

=== Bước 2: Load model & build SFTTrainer ===
Load tokenizer & model: Qwen/Qwen3-0.6B


...

Parameter 'function'=<function SFTTrainer._prepare_dataset.<locals>.tokenize_fn at 0x7ef35a8134c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only shown once. Subsequent hashing failures won't be shown.


Tokenizing test dataset:   0%|          | 0/2507 [00:00<?, ? examples/s]

Truncating test dataset:   0%|          | 0/2507 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Đã lưu eval_metrics vào: /kaggle/working/qwen3_qa_model/eval_test.json

=== Test metrics ===
{'eval_loss': 0.5347903966903687, 'eval_runtime': 113.7003, 'eval_samples_per_second': 22.049, 'eval_steps_per_second': 2.762, 'eval_entropy': 0.5817697042492544, 'eval_num_tokens': 7811546.0, 'eval_mean_token_accuracy': 0.8840812784471329, 'epoch': 2.0, 'perplexity': 1.7070903929561196, 'bleu': 62.39244812908962}


In [11]:
!rm -rf /kaggle/working/qwen3_qa_model/checkpoint-1269
!rm -rf /kaggle/working/qwen3_qa_model/checkpoint-2538
!zip -r /kaggle/working/qwen3_qa_model.zip /kaggle/working/qwen3_qa_model


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/qwen3_qa_model/ (stored 0%)
  adding: kaggle/working/qwen3_qa_model/training_args.bin (deflated 51%)
  adding: kaggle/working/qwen3_qa_model/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/qwen3_qa_model/runs/ (stored 0%)
  adding: kaggle/working/qwen3_qa_model/runs/Nov20_15-00-00_e62f7eaab2de/ (stored 0%)
  adding: kaggle/working/qwen3_qa_model/runs/Nov20_15-00-00_e62f7eaab2de/events.out.tfevents.1763656351.e62f7eaab2de.48.1 (deflated 30%)
  adding: kaggle/working/qwen3_qa_model/runs/Nov20_15-00-00_e62f7eaab2de/events.out.tfevents.1763650820.e62f7eaab2de.48.0 (deflated 66%)
  adding: kaggle/working/qwen3_qa_model/chat_template.jinja (deflated 76%)
  adding: kaggle/working/qwen3_qa_model/train_metrics.json (deflated 37%)
  adding: kaggle/working/qwen3_qa_model/special_tokens_map.json (deflated 69%)
  adding: kaggle/working/qwen3_qa_model/tokenizer_config.json (deflated 90%)
  adding: kaggle/working/qwen3_qa_model/vocab.json (deflated 61%)
  adding: kaggle/working/qwen3_qa_model/generation_config.json (deflated 32%)
  adding: kaggle/working/qwen3_qa_model/eval_test.json (deflated 40%)
  adding: kaggle/working/qwen3_qa_model/added_to